# Preparation stuff

## Connect to Drive

In [1]:
connect_to_drive = False

In [2]:
#Run command and authorize by popup --> other window
if connect_to_drive:
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)

## Install packages

In [3]:
if connect_to_drive:
    #Install FS code
    !pip install  --upgrade --force-reinstall git+https://github.com/federicosiciliano/easy_lightning.git

    !pip install pytorch_lightning

## IMPORTS

In [4]:
#Put all imports here
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
#import pickle
import os
import sys
#import cv2
import csv
import torch

## Define paths

In [5]:
#every path should start from the project folder:
project_folder = "../"
if connect_to_drive:
    project_folder = "/content/gdrive/Shareddrives/<SharedDriveName>" #Name of SharedDrive folder
    #project_folder = "/content/gdrive/MyDrive/<MyDriveName>" #Name of MyDrive folder

#Config folder should contain hyperparameters configurations
cfg_folder = os.path.join(project_folder,"cfg")

#Data folder should contain raw and preprocessed data
data_folder = os.path.join(project_folder,"data")
raw_data_folder = os.path.join(data_folder,"raw")
processed_data_folder = os.path.join(data_folder,"processed")

#Source folder should contain all the (essential) source code
source_folder = os.path.join(project_folder,"src")

#The out folder should contain all outputs: models, results, plots, etc.
out_folder = os.path.join(project_folder,"out")
img_folder = os.path.join(out_folder,"img")

## Import own code

In [6]:
#To import from src:

#attach the source folder to the start of sys.path
sys.path.insert(0, project_folder)

#import from src directory
# from src import ??? as additional_module
import easy_rec as additional_module #REMOVE THIS LINE IF IMPORTING OWN ADDITIONAL MODULE

import easy_exp, easy_rec, easy_torch #easy_data

/home/filippobetello/.conda/envs/recsys/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-21 17:19:11,711	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-21 17:19:11,800	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


# MAIN

## Train

### Data

In [7]:
cfg = easy_exp.cfg.load_configuration("config_rec")

In [8]:
#cfg["data_params"]["test_sizes"] = [cfg["data_params.dataset_params.out_seq_len.val"],cfg["data_params.dataset_params.out_seq_len.test"]]

data_params = cfg["data_params"].copy()
data_params["data_folder"] = raw_data_folder

data, maps = easy_rec.data_generation_utils.preprocess_dataset(**data_params)

Ratings data already exists. Skip pre-processing
Filtering by minimum number of users per item: 5
Filtering by minimum number of items per user: 5
Densifying index
Splitting: leave_n_out


In [9]:
item_infos = easy_rec.data_generation_utils.load_item_info('../data/raw', "ml-100k")
print(len(item_infos))
print(item_infos.keys())
print(item_infos[['movie_id', 'movie_title']].head(3))

1682
Index(['movie_id', 'movie_title', 'release date', 'video release date',
       'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children's',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'],
      dtype='object')
   movie_id        movie_title
0         1   Toy Story (1995)
1         2   GoldenEye (1995)
2         3  Four Rooms (1995)


In [10]:
data['train_sid']

array([list([167, 171, 164, 155, 165, 195, 186, 249, 126, 14, 117, 109, 180, 1, 245, 247, 256, 50, 248, 252, 261, 93, 223, 123, 19, 136, 122, 145, 7, 234, 15, 244, 24, 263, 125, 259, 236, 13, 25, 120, 250, 235, 239, 118, 129, 47, 65, 189, 31, 28, 39, 114, 52, 237, 69, 182, 11, 198, 95, 160, 178, 60, 83, 134, 64, 98, 22, 26, 162, 201, 8, 89, 213, 181, 48, 99, 174, 191, 159, 127, 179, 184, 142, 68, 55, 203, 56, 96, 79, 81, 150, 211, 23, 70, 84, 196, 190, 94, 183, 133, 206, 144, 187, 185, 97, 36, 158, 85, 143, 17, 173, 251, 105, 147, 108, 146, 219, 242, 106, 121, 104, 246, 107, 119, 45, 267, 266, 258, 260, 262, 10, 149, 233, 71, 42, 92, 175, 91, 192, 216, 176, 215, 193, 73, 59, 41, 132, 194, 169, 217, 212, 156, 222, 226, 27, 80, 230, 67, 199, 4, 214, 163, 2, 205, 77, 135, 53, 46, 40, 152, 268, 253, 115, 172, 210, 228, 154, 202, 62, 90, 218, 166, 161, 61, 229, 35, 112, 264, 57, 49, 30, 232, 130, 151, 82, 140, 72, 157, 33, 197, 113, 224, 21, 88, 148, 238, 110, 101, 103, 43, 34, 29, 131, 204

In [11]:
item_infos['movie_title'].values

array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
       'Sliding Doors (1998)', 'You So Crazy (1994)',
       'Scream of Stone (Schrei aus Stein) (1991)'],
      shape=(1682,), dtype=object)

In [12]:
datasets = easy_rec.rec_torch.prepare_rec_datasets(data,**data_params["dataset_params"])

In [13]:
for i in datasets['train']:
    print(i)

{'sid': [167, 171, 164, 155, 165, 195, 186, 249, 126, 14, 117, 109, 180, 1, 245, 247, 256, 50, 248, 252, 261, 93, 223, 123, 19, 136, 122, 145, 7, 234, 15, 244, 24, 263, 125, 259, 236, 13, 25, 120, 250, 235, 239, 118, 129, 47, 65, 189, 31, 28, 39, 114, 52, 237, 69, 182, 11, 198, 95, 160, 178, 60, 83, 134, 64, 98, 22, 26, 162, 201, 8, 89, 213, 181, 48, 99, 174, 191, 159, 127, 179, 184, 142, 68, 55, 203, 56, 96, 79, 81, 150, 211, 23, 70, 84, 196, 190, 94, 183, 133, 206, 144, 187, 185, 97, 36, 158, 85, 143, 17, 173, 251, 105, 147, 108, 146, 219, 242, 106, 121, 104, 246, 107, 119, 45, 267, 266, 258, 260, 262, 10, 149, 233, 71, 42, 92, 175, 91, 192, 216, 176, 215, 193, 73, 59, 41, 132, 194, 169, 217, 212, 156, 222, 226, 27, 80, 230, 67, 199, 4, 214, 163, 2, 205, 77, 135, 53, 46, 40, 152, 268, 253, 115, 172, 210, 228, 154, 202, 62, 90, 218, 166, 161, 61, 229, 35, 112, 264, 57, 49, 30, 232, 130, 151, 82, 140, 72, 157, 33, 197, 113, 224, 21, 88, 148, 238, 110, 101, 103, 43, 34, 29, 131, 204, 20

In [14]:
collator_params = cfg["data_params"]["collator_params"].copy()
collator_params["num_items"] = np.max(list(maps["sid"].values()))

In [15]:
collators = easy_rec.rec_torch.prepare_rec_collators(**collator_params)

In [16]:
loader_params = cfg["model"]["loader_params"].copy()
loaders = easy_rec.rec_torch.prepare_rec_data_loaders(datasets, **loader_params, collate_fn=collators)

In [17]:
for _ in range(10):
    cnt = 0
    cnt2 = 0
    app = []
    for i in loaders['train']:
        bs, sl = i['in_sid'].shape
        for j in range(bs):
            while int(i['in_sid'][j][cnt]) == 0:
                cnt+=1
                continue
            
            app.append(cnt)
            cnt=0
            if int(i['in_sid'][j][0]) == 0:
                cnt2+=1
        


    # print(app)
    # print(cnt2)
    m = sum(app)/len(app)
    print(m)

tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   

tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   

NameError: Caught NameError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/filippobetello/.conda/envs/recsys/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/home/filippobetello/.conda/envs/recsys/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
  File "/hdd2/filippobetello/projects/environmental_sampling/ntb/../easy_rec/rec_torch.py", line 392, in __call__
    out = super().__call__(batch)
  File "/hdd2/filippobetello/projects/environmental_sampling/ntb/../easy_rec/rec_torch.py", line 172, in __call__
    out = self.pair_input_output(out, seq_lens)
  File "/hdd2/filippobetello/projects/environmental_sampling/ntb/../easy_rec/rec_torch.py", line 331, in pair_input_output
    aaa
NameError: name 'aaa' is not defined


In [ ]:
a = [876,  50, 309, 249, 285, 180, 513, 193, 653, 580, 472, 430, 213, 198, 622, 521, 211, 203]

In [ ]:
for i in data['train_sid']:
    if a in i:
        print(i)
        break


In [ ]:
i['in_sid'][j]

tensor([876,  50, 309, 249, 285, 180, 513, 193, 653, 580, 472, 430, 213, 198,
        622, 521, 211, 203,   0, 876,  50, 309, 249, 285, 180, 513, 193, 653,
        580, 472, 430, 213, 198, 622, 521, 211, 203,   0, 876,  50, 309, 249,
        285, 180, 513, 193, 653, 580, 472, 430, 213, 198, 622, 521, 211, 203,
          0, 876,  50, 309, 249, 285, 180, 513, 193, 653, 580, 472, 430, 213,
        198, 622, 521, 211, 203,   0, 876,  50, 309, 249, 285, 180, 513, 193,
        653, 580, 472, 430, 213, 198, 622, 521, 211, 203,   0, 876,  50, 309,
        249, 285, 180, 513, 193, 653, 580, 472, 430, 213, 198, 622, 521, 211,
        203,   0, 876,  50, 309, 249, 285, 180, 513, 193, 653, 580, 472, 430,
        213, 198, 622, 521, 211, 203,   0, 876,  50, 309, 249, 285, 180, 513,
        193, 653, 580, 472, 430, 213, 198, 622, 521, 211, 203,   0, 876,  50,
        309, 249, 285, 180, 513, 193, 653, 580, 472, 430, 213, 198, 622, 521,
        211, 203,   0, 876,  50, 309, 249, 285, 180, 513, 193, 6

In [ ]:
# 111.9  --> padding di media for ml-100k su seq len di 200
# 92.8 --> padding di media for ml-1m su seq len di 200
# 47.4 --> padding di media for amazon beauty su seq len di 50




SyntaxError: invalid decimal literal (931989014.py, line 1)

In [ ]:
# put something intelligent instead of padding


cnt = 0
cnt2 = 0
for i in loaders['train']:
    bs, sl = i['in_sid'].shape
    for j in range(bs):
        while int(i['in_sid'][j][cnt]) == 0:
            cnt+=1
            continue
        cnt-=4 # perché poi con val e test aggiungo 2 numeri e mi serve un poquito di spazio per padding in ogni caso? check repeated padding strategy

        if cnt> 0: # here start stuff to copy the last part of the sequence so it will be seen twice
            app = 0 
            while cnt>0:
                i['in_sid'][j][app] = i['in_sid'][j][-cnt]
                cnt-=1
                app+=1

for i in loaders['val']:
    bs, sl = i['in_sid'].shape
    for j in range(bs):
        while int(i['in_sid'][j][cnt]) == 0:
            cnt+=1
            continue
        cnt-=4 # perché poi con val e test aggiungo 2 numeri e mi serve un poquito di spazio per padding in ogni caso? check repeated padding strategy

        if cnt> 0: # here start stuff to copy the last part of the sequence so it will be seen twice
            app = 0 
            while cnt>0:
                i['in_sid'][j][app] = i['in_sid'][j][-cnt]
                cnt-=1
                app+=1


for i in loaders['test']:
    bs, sl = i['in_sid'].shape
    for j in range(bs):
        while int(i['in_sid'][j][cnt]) == 0:
            cnt+=1
            continue
        cnt-=4 # perché poi con val e test aggiungo 2 numeri e mi serve un poquito di spazio per padding in ogni caso? check repeated padding strategy

        if cnt> 0: # here start stuff to copy the last part of the sequence so it will be seen twice
            app = 0 
            while cnt>0:
                i['in_sid'][j][app] = i['in_sid'][j][-cnt]
                cnt-=1
                app+=1
        
    

